# scenario-based forecast model

## loading data form bigquery

In [1]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json("/Users/sarah/Desktop/git/Spotify FP&A/service_acc_key.json")
tables = list(client.list_tables("fp-and-a.dbt_staging"))
print(tables)


[<google.cloud.bigquery.table.TableListItem object at 0x10f2e2f30>, <google.cloud.bigquery.table.TableListItem object at 0x10f20f740>, <google.cloud.bigquery.table.TableListItem object at 0x10e8b69f0>, <google.cloud.bigquery.table.TableListItem object at 0x10f8bbad0>, <google.cloud.bigquery.table.TableListItem object at 0x10f8bbf20>, <google.cloud.bigquery.table.TableListItem object at 0x10f8bbd40>]


In [2]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json("/Users/sarah/Desktop/git/Spotify FP&A/service_acc_key.json")

query = """
SELECT *
FROM `fp-and-a.dbt_fct.fct_content_performance`
"""

base_df = client.query(query).to_dataframe()


/Users/sarah/anaconda3/envs/sarah_env/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


## transforming data for analysis

In [3]:
df = base_df.copy()
df.head()

,content_id,user_id,stream_date,streams,impressions_per_stream,estimated_ad_revenue,total_royalty_cost,gross_margin,cpm,royalty_per_stream,content_type
0,3JW6QqeTrwavHlUiC2ka0g,69a5a17e-2ae0-48fc-9d5a-31f0decc918a,2024-01-04,1,1,0.01,0.002,0.008,10,0.002,podcast
1,2jCpGtOzrs8NwQC35xaUwK,c49dfcd9-b176-4388-8e49-5179138d23b6,2024-01-04,1,1,0.01,0.002,0.008,10,0.002,podcast
2,7c181EiST5DhX4Zh2kt7Tq,fa54eaea-dd62-4cf8-a9de-ea54a402f8c5,2024-01-08,1,1,0.01,0.002,0.008,10,0.002,podcast
3,730sncvN1ArXcamdiqvC9M,55f9ff84-1734-4d79-b01c-f6b5c8553747,2024-01-12,1,1,0.01,0.002,0.008,10,0.002,podcast
4,6Dp3mz0m4EFX0lVN4BFbQ1,171c43f3-84dd-4968-a240-85f5ac47f87d,2024-01-25,1,1,0.01,0.002,0.008,10,0.002,podcast


In [15]:
df.groupby("content_type")["impressions_per_stream"].sum()

content_type
music       64361
podcast    189798
Name: impressions_per_stream, dtype: Int64

In [ ]:
result = (
    df.groupby("content_type")
      .agg({
          "streams": "sum",
          "estimated_ad_revenue": "sum",
          "gross_margin": "sum",
          "cpm": "median",
          "impressions_per_stream": "sum",
          "royalty_per_stream": "median"
      })
      .reset_index()
)
result["avg_cpm"] = result["cpm"]
result['impressions_per_stream'] = (
    result['estimated_ad_revenue'] / 
    result['streams'] / 
    result['cpm'] * 1000
)

result

,content_type,streams,estimated_ad_revenue,gross_margin,cpm,impressions_per_stream,royalty_per_stream,avg_cpm
0,music,111814,1120.425,673.169,5.0,2.004087,0.004,5.0
1,podcast,331470,6646.260,5983.320,10.0,2.005086,0.002,10.0


# scenario based analysis

In [17]:
import pandas as pd

# scenarios
stream_changes = [0.10, 0.20]
cpm_changes = [0.05, 0.10]

scenarios = []

for s in stream_changes:
    for c in cpm_changes:
        temp = result.copy()
        
        temp["streams_new"] = temp["streams"] * (1 + s)
        temp["cpm_new"] = temp["avg_cpm"] * (1 + c)
        
        temp["revenue_new"] = (
            temp["streams_new"]
            * temp["impressions_per_stream"]
            * temp["cpm_new"] / 1000
        )
        
        temp["cost_new"] = (
            temp["streams_new"]
            * temp["royalty_per_stream"]
        )
        
        temp["margin_new"] = temp["revenue_new"] - temp["cost_new"]
        
        temp["stream_change"] = s
        temp["cpm_change"] = c
        
        scenarios.append(temp)

scenario_df = pd.concat(scenarios)
scenario_df.head()

,content_type,streams,estimated_ad_revenue,gross_margin,cpm,impressions_per_stream,royalty_per_stream,avg_cpm,streams_new,cpm_new,revenue_new,cost_new,margin_new,stream_change,cpm_change
0,music,111814,1120.425,673.169,5.0,2.004087,0.004,5.0,122995.4,5.25,1294.090875,491.9816,802.109275,0.1,0.05
1,podcast,331470,6646.260,5983.320,10.0,2.005086,0.002,10.0,364617.0,10.5,7676.4303,729.234,6947.1963,0.1,0.05
0,music,111814,1120.425,673.169,5.0,2.004087,0.004,5.0,122995.4,5.5,1355.71425,491.9816,863.73265,0.1,0.10
1,podcast,331470,6646.260,5983.320,10.0,2.005086,0.002,10.0,364617.0,11.0,8041.9746,729.234,7312.7406,0.1,0.10
0,music,111814,1120.425,673.169,5.0,2.004087,0.004,5.0,134176.8,5.25,1411.7355,536.7072,875.0283,0.2,0.05


Scenario analysis shows a 10% increase in podcast streams & a 5% cpm increase could yield an additional $1,030 in ad revenue, reinforcing the case for Business & Comedy podcast investment(as seen in PowerBI Dashboard). However, a 20% increase in streams with a 10% CPM boost could generate an extra $1395, suggesting that aggressive growth strategies in these genres may be highly profitable.